In [63]:
import tensorflow as tf

In [117]:
from util.input import inputs

In [118]:
import os
import itertools
import tensorflow as tf
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd
import cv2
import matplotlib.pyplot as plt

def make_pickle(imgs, img_names, type):
    img_label = {'images': imgs, 'labels': img_names}
    pickle.dump(img_label, open("data/%s.pkl" % type, "wb"))


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

def make_tfrecords(images, labels, type):

    num_examples = labels.shape[0]

    if images.shape[0] != num_examples:
        raise ValueError("Images size %d does not match label size %d." %
                         (images.shape[0], num_examples))
    rows = images.shape[1]
    cols = images.shape[2]
    depth = images.shape[3]

    directory = "."

    filename = os.path.join(directory, type + '.tfrecords')
    print('Writing', filename)
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(len(labels)):
        image_raw = images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(rows),
            'width': _int64_feature(cols),
            'depth': _int64_feature(depth),
            'label': _float_feature(float(labels[index])),
            'image_raw': _bytes_feature(image_raw)}))
        writer.write(example.SerializeToString())

In [119]:
CHANNEL = 3
HEIGHT = 60
WEIGHT = 80

In [120]:
def label0():
    return tf.constant(0)
def label1():
    return tf.constant(1)
def label2():
    return tf.constant(2)
def label3():
    return tf.constant(3)
def label4():
    return tf.constant(4)
def default():
    return tf.constant(-1)

In [157]:
def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
        serialized_example,
        # Defaults are not specified since both keys are required.
        features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            # 'angle': tf.FixedLenFeature([], tf.float32),
#             'label': tf.FixedLenFeature([], tf.int64),
            'label': tf.FixedLenFeature([], tf.float32),
            'img_name': tf.FixedLenFeature([], tf.string)
        })

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    img_name = tf.decode_raw(features['img_name'], tf.uint8)
#



#     img_name = tf.cast(img_name, tf.int64)
#     img_name = None
    image.set_shape([HEIGHT * WEIGHT * CHANNEL])
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    image = tf.reshape(image, [HEIGHT, WEIGHT, CHANNEL])
    # image = tf.image.resize_images(image, tf.pack(tf.constant(60, dtype=tf.int32), tf.constant(80, dtype=tf.int32)))
    
    # preprocessing
    # image = tf.image.rgb_to_grayscale(image)
    # image = tf.image.per_image_standardization(image)
    # image = tf.image.per_image_whitening(image)

    # Convert label from a scalar uint8 tensor to an int32 scalar.
#     angle = tf.cast(features['label'], tf.float32)


#     label = tf.cond(tf.less(angle, tf.constant(-0.03966)), label0,
#             lambda: tf.cond(tf.less(angle, tf.constant(-0.00698)), label1,
#                             lambda: tf.cond(tf.less(angle, tf.constant(0.01266)), label2,
#                                             lambda: tf.cond(tf.less(angle, tf.constant(0.04189)), label3, label4))))

#     angle = tf.reshape(angle, [1])

    label = None
    angle = None

    # label = None
    # label = tf.cond(tf.equal(label, 0), lambda: tf.convert_to_tensor(0), lambda: tf.convert_to_tensor(1))

    return image, angle, label, img_name

In [158]:
TRAIN_DIR = "data"

In [159]:
def aggregate_dataset(direction, dataset, dataset_subset='all'):

    """
        direction = ['left', 'center', 'right']
        dataset = ['original', 'flip', 'contrast', 'flip_contrast']
        dataset_subset = 'all' or ['train','val']

        return {}
    """
    train_lst = []
    test_lst = []
    validation_lst = []

    def get_subset_lst(subset):
        subste_list = []
        for a, b in list(itertools.product(direction, dataset)):
            folders = [folder for folder in os.listdir(TRAIN_DIR) if folder.startswith("udacity")]
            for folder in folders:

                try:
                    dir = TRAIN_DIR + "/" + folder + "/%s/%s" % (a, b)
                    train = [dir + "/" + file for file in os.listdir(dir) if subset in file]
                    subste_list += train
                except:
                    pass
        return [subset, subste_list]

    train_lst = np.array(train_lst)
    test_lst = np.array(test_lst)
    validation_lst = np.array(validation_lst)

    if dataset_subset is 'all':
        dataset_subset = ['train', 'test', 'validation']

    total_dict = {}
    for subset in dataset_subset:
        total_dict[get_subset_lst(subset)[0]] = get_subset_lst(subset)[1]

    return total_dict

In [160]:

filename = aggregate_dataset(['center'], ['contrast'], ['train'])['train']
filename = ["data/submission_test.tfrecords"]

In [161]:
num_epochs = None
batch_size = 100
with tf.name_scope('input'):
    filename_queue = tf.train.string_input_producer(
        filename, num_epochs=num_epochs)

    image, angle, label, img_name = read_and_decode(filename_queue)
    images = []
    img_names = []
    with tf.Session() as sess:
        # Start populating the filename queue.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        tmp = []
        for i in range(100):
            # Retrieve a single instance:
            a = sess.run([img_name])
#             if c in tmp:
#                 break
#             tmp.append(c)
            images.append(a)
#             img_names.append(b)

        coord.request_stop()
        coord.join(threads)
        print('done!!')

InvalidArgumentError: Name: <unknown>, Feature: label is required but could not be found.
	 [[Node: input_36/ParseSingleExample/ParseExample/ParseExample = ParseExample[Ndense=3, Nsparse=0, Tdense=[DT_STRING, DT_STRING, DT_FLOAT], dense_shapes=[[], [], []], sparse_types=[], _device="/job:localhost/replica:0/task:0/cpu:0"](input_36/ParseSingleExample/ExpandDims, input_36/ParseSingleExample/ParseExample/ParseExample/names, input_36/ParseSingleExample/ParseExample/ParseExample/dense_keys_0, input_36/ParseSingleExample/ParseExample/ParseExample/dense_keys_1, input_36/ParseSingleExample/ParseExample/ParseExample/dense_keys_2, input_36/ParseSingleExample/ParseExample/Const, input_36/ParseSingleExample/ParseExample/Const_1, input_36/ParseSingleExample/ParseExample/Const_2)]]

Caused by op 'input_36/ParseSingleExample/ParseExample/ParseExample', defined at:
  File "/Users/daniel/anaconda3/envs/tensorflow2/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  Fil

In [53]:
img_names[999]

array([-0.00174533], dtype=float32)

In [47]:
np.concatenate(img_names)

array([-0.00872665,  0.00174533, -0.0106064 , -0.01047198, -0.00174533,
       -0.00174533,  0.04837419, -0.03490658,  0.02443461,  0.00375849,
        0.01472643,  0.00022914,  0.12566371, -0.01396263,  0.05078869,
       -0.01570796,  0.03675999,  0.03316126, -0.01047198,  0.04184481,
       -0.01491667, -0.04686459, -0.04537856, -0.071329  , -0.03141593,
        0.05811516,  0.037611  ,  0.01047198, -0.01396263,  0.08028515,
        0.01964896, -0.01912297, -0.00174533, -0.00729858, -0.01919862,
        0.07200993,  0.07330383,  0.02792527, -0.02268928, -0.02792527,
       -0.02094395,  0.00872665, -0.00523599,  0.07412998, -0.00375505,
       -0.03316126, -0.09250245,  0.01396263,  0.01896811,  0.0648128 ,
       -0.00174533,  0.0175952 , -0.00537863,  0.05149613, -0.01460221,
       -0.00349066, -0.00698132, -0.02268928,  0.00698132,  0.00872665,
       -0.06766396, -0.01745329,  0.00174533,  0.0346966 , -0.00872665,
        0.00872665, -0.00140603, -0.06283186, -0.01919862,  0.00

In [20]:
filenames = aggregate_dataset(['center'], ['original'], ['train'])['train']

In [21]:
filenames

['data/udacity-dataset-2/center/original/train_0.tfrecords']

In [22]:
num_epochs = None
batch_size = 100
one_hot_labels = False
with tf.name_scope('input'):
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs)

    image, angle, label, img_name = read_and_decode(filename_queue)
    if one_hot_labels:
        print("one_hot_labels")
        label = tf.one_hot(label, NUM_CLASSES, dtype=tf.int32)
        images, sparse_labels = tf.train.shuffle_batch(
            [image, label], batch_size=batch_size, num_threads=4,
            capacity=1000 + 3 * batch_size,
            # Ensures a minimum amount of shuffling of examples.
            min_after_dequeue=1000)
#         return images, sparse_labels
    else:
        pass

In [23]:
images, sparse_angles = tf.train.shuffle_batch(
                        [image, angle], batch_size=batch_size, num_threads=4,
                        capacity=1000 + 3 * batch_size,
                        # Ensures a minimum amount of shuffling of examples.
                        min_after_dequeue=1000)

In [30]:
images = []
angles = []

num_epochs = None
batch_size = 100
one_hot_labels = False
with tf.name_scope('input'):
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs)

    image, angle, label, img_name = read_and_decode(filename_queue)
    if one_hot_labels:
        print("one_hot_labels")
        label = tf.one_hot(label, NUM_CLASSES, dtype=tf.int32)
        images, sparse_labels = tf.train.shuffle_batch(
            [image, label], batch_size=batch_size, num_threads=4,
            capacity=1000 + 3 * batch_size,
            # Ensures a minimum amount of shuffling of examples.
            min_after_dequeue=1000)
#         return images, sparse_labels
    else:
        print('here')
        with tf.Session() as sess:
            # Start populating the filename queue.
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(coord=coord)

        #     images, sparse_angles = tf.train.shuffle_batch(
        #         [image, angle], batch_size=batch_size, num_threads=4,
        #         capacity=1000 + 3 * batch_size,
        #         # Ensures a minimum amount of shuffling of examples.
        #         min_after_dequeue=1000)

            for i in range(batch_size):
                # Retrieve a single instance:
                img = sess.run([image])
                print(img)
        #         if label == 0:

        #             if angle < -0.03966:
        #                 images.append(img)
        #                 angles.append(ang)
        #         elif label == 1:
        #             if -0.03966 <= ang and ang < -0.00698:
        #                 images.append(img)
        #                 angles.append(ang)
        #         elif label == 2:
        #             if -0.00698 <= ang and ang < 0.01266:
        #                 images.append(img)
        #                 angles.append(ang)
        #         elif label == 3:
        #             if 0.01266 <= ang and ang < 0.04189:
        #                 images.append(img)
        #                 angles.append(ang)
        #         elif label == 4:
        #             if 0.04189 <= ang:
        #                 images.append(img)
        #                 angles.append(ang)

            coord.request_stop()
            coord.join(threads)

here


InvalidArgumentError: Name: <unknown>, Feature: label.  Data types don't match. Data type: int64Expected type: float
	 [[Node: input_5/ParseSingleExample/ParseExample/ParseExample = ParseExample[Ndense=2, Nsparse=0, Tdense=[DT_STRING, DT_FLOAT], dense_shapes=[[], []], sparse_types=[], _device="/job:localhost/replica:0/task:0/cpu:0"](input_5/ParseSingleExample/ExpandDims, input_5/ParseSingleExample/ParseExample/ParseExample/names, input_5/ParseSingleExample/ParseExample/ParseExample/dense_keys_0, input_5/ParseSingleExample/ParseExample/ParseExample/dense_keys_1, input_5/ParseSingleExample/ParseExample/Const, input_5/ParseSingleExample/ParseExample/Const_1)]]

Caused by op 'input_5/ParseSingleExample/ParseExample/ParseExample', defined at:
  File "/Users/daniel/anaconda3/envs/tensorflow2/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/daniel/anaconda3/envs/tensorflow2/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_glob

In [18]:
images

<tf.Tensor 'shuffle_batch:0' shape=(100, 60, 80, 3) dtype=float32>

In [7]:
image, angle, label, img_name = read_and_decode(filename_queue)

NameError: name 'filename_queue' is not defined

In [ ]:
images, angles = inputs(train_dir, True, batch_size, num_batches, label, one_hot_labels=False)   